In [1]:
#importing libraries

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import neattext.functions as nfx
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel
from collections import Counter


Bad key "text.kerning_factor" on line 4 in
C:\Users\Atul\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [28]:
#loading dataset
df=pd.read_csv('Resume_Dataset_Sample.csv')

In [29]:
df.head()

,Unnamed: 0,filename,resume
0,0,DanahNielsen_26409,danah nielsen hostess albuquerque nm seek...
1,1,ChristopherBrown_30650,christopher brown west memphis ar to obta...
2,2,MinnesotaStateParksWorkerSecurityRangerSeasona...,minnesota state parks worker security ranger...
3,3,NaveenNetDeveloper_46188,naveen net developer java developer bcbsm ...
4,4,TimSchannauer_22646,tim schannauer electrician helper lane valente...


In [30]:
df['resume'] = df['resume'].str.replace('()','').str.replace('-','').str.replace('.','')

In [31]:
df['resume'] = df['resume'].str.lower()

In [32]:
df.head()

,Unnamed: 0,filename,resume
0,0,DanahNielsen_26409,danah nielsen hostess albuquerque nm seek...
1,1,ChristopherBrown_30650,christopher brown west memphis ar to obta...
2,2,MinnesotaStateParksWorkerSecurityRangerSeasona...,minnesota state parks worker security ranger...
3,3,NaveenNetDeveloper_46188,naveen net developer java developer bcbsm ...
4,4,TimSchannauer_22646,tim schannauer electrician helper lane valente...


In [33]:
df['resume'] = df['resume'].map(str)

In [34]:
df['resume'] = df['resume'].apply(nfx.remove_numbers).apply(nfx.remove_stopwords).apply(nfx.remove_special_characters)

In [35]:
import nltk 
nltk.download('punkt') 
nltk.download('averaged_perceptron_tagger') 
nltk.download('wordnet') 
  
  
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 
  
from nltk.corpus import stopwords 
nltk.download('stopwords') 
stop_words = set(stopwords.words('english')) 
  
VERB_CODES = {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'} 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Atul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Atul\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Atul\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Atul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
len(df) 
  
finaldata = df[["filename", "resume"]]          # Required columns - Title and combined data  
finaldata = finaldata.set_index('filename')    # Setting the job title title as index 
  
finaldata.head(10) 
finaldata["resume"][0] 

'danah nielsen hostess albuquerque nm seeking position challenge provide ongoing experiences organization utilizing skills assisting development additional skills professional advancement work experience hostess hop august greeted customers monitored open dining sections cleaned tables managed guest waiting list answering telephone booking reservations center representative september march received inbound calls assist customers problems found resolutions ensure customer satisfaction outbound calls provide customer satisfaction service purchases provided prompt customer service follow appointments necessary server kelly brew pub august september took orders efficiently served food drinks removed dinnerware table responsibly served alcohol state local requirements managed money transactions appropriately completed opening closing procedures dictated establishment time sheet auditor consumer direct personal care october july audited timesheets submitted caregivers ensure completeness acc

In [37]:
finaldata.shape

(571, 1)

In [38]:
#takes few mins
def preprocess_sentences(text): 
  text = text.lower() 
  temp_sent =[] 
  words = nltk.word_tokenize(text) 
  tags = nltk.pos_tag(words) 
  for i, word in enumerate(words): 
      if tags[i][1] in VERB_CODES:  
          lemmatized = lemmatizer.lemmatize(word, 'v') 
      else: 
          lemmatized = lemmatizer.lemmatize(word) 
      if lemmatized not in stop_words and lemmatized.isalpha(): 
          temp_sent.append(lemmatized) 
          
  finalsent = ' '.join(temp_sent) 
  finalsent = finalsent.replace("n't", " not") 
  finalsent = finalsent.replace("'m", " am") 
  finalsent = finalsent.replace("'s", " is") 
  finalsent = finalsent.replace("'re", " are") 
  finalsent = finalsent.replace("'ll", " will") 
  finalsent = finalsent.replace("'ve", " have") 
  finalsent = finalsent.replace("'d", " would") 
  return finalsent 
  
finaldata["resume_processed"]= finaldata["resume"].apply(preprocess_sentences) 
finaldata.head() 

,resume,resume_processed
filename,,
DanahNielsen_26409,danah nielsen hostess albuquerque nm seeking p...,danah nielsen hostess albuquerque nm seek posi...
ChristopherBrown_30650,christopher brown west memphis ar obtain posit...,christopher brown west memphis ar obtain posit...
MinnesotaStateParksWorkerSecurityRangerSeasonalParttime_32908,minnesota state parks worker security ranger s...,minnesota state park worker security ranger se...
NaveenNetDeveloper_46188,naveen net developer java developer bcbsm mi d...,naveen net developer java developer bcbsm mi d...
TimSchannauer_22646,tim schannauer electrician helper lane valente...,tim schannauer electrician helper lane valente...


In [ ]:
#we can use this according to data that how much clean final data is
#import re
#remove_less = lambda x: re.sub(r'\b\w{​​1,4}​​\b', '', x)
#finaldata["resume_processed"]= finaldata["resume"].apply(remove_less)

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer 
  
# Vectorizing pre-processed combo using TF-IDF 
tfidfvec = TfidfVectorizer() 
tfidf_resume = tfidfvec.fit_transform((finaldata["resume_processed"])) 

In [40]:
tfidfvec

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [41]:
tfidf_resume

<571x11526 sparse matrix of type '<class 'numpy.float64'>'
	with 72185 stored elements in Compressed Sparse Row format>

In [42]:
tfidf_resume.shape

(571, 11526)

In [43]:
# Finding cosine similarity between vectors 
from sklearn.metrics.pairwise import cosine_similarity 
cos_sim = cosine_similarity(tfidf_resume, tfidf_resume)

In [50]:
# Storing indices of the data 
indices = pd.Series(finaldata.index) 
  
def recommendations(title, cosine_sim = cos_sim): 
    recommended_resume = [] 
    index = indices[indices == title].index[0] 
    similarity_scores = pd.Series(cosine_sim[index]).sort_values(ascending = False) 
    top_10_resume = list(similarity_scores.iloc[1:11].index) 
    for i in top_10_resume: 
        recommended_resume.append(list(finaldata.index)[i]) 
    return recommended_resume 

# Given Resume to similar Resume

In [56]:
#recommendation sample 1
recommendations("ChristopherBrown_30650")

['ShiftLeader_43124',
 'ProductionSupervisor_48124',
 'AlyshaWynn_4697',
 'MachineOperator_15529',
 'LeonardShaw_32044',
 'CookPrep_15851',
 'JobSeeker_10962',
 'OperationsProcessor_30149',
 'LindaFranklin_4611',
 'Teammember_36420']

In [57]:
#recommendation sample 2
recommendations("LindaFranklin_4611")

['AlyshaWynn_4697',
 'JobSeeker_10962',
 'PainterHelper_36178',
 'LandscapeWorker_47840',
 'MembershipRepresentative_12507',
 'Server_3899',
 'ChristopherBrown_30650',
 'RoshellWest_37940',
 'PatientCareTechnician_3358',
 'MeganBurden_21795']

In [58]:
#recommendation sample 3
recommendations("TimSchannauer_22646")

['HoracePalmer_7845',
 'RonaldHAynes_4727',
 'DavidMajeski_26220',
 'FarahDieu_20146',
 'JosephQuirino_48764',
 'BenCole_41560',
 'AlexTucker_13146',
 'VanjaTomovic_10917',
 'SchoolBusDriverSubDriver_44',
 'StockAssociate_44849']

In [59]:
#recommendation sample 4
recommendations("Teammember_36420")

['AdministrativeAssistantRegionalSalesAssistant_15895',
 'DepartmentManager_46542',
 'ChristopherBrown_30650',
 'HousekeeperLaundry_14961',
 'Server_2184',
 'Caregiver_49697',
 'CrewMember_11837',
 'JanahBrown_38437',
 'LeonardShaw_32044',
 'SaravSundar_48772']

In [60]:
#recommendation sample 5
recommendations("DanahNielsen_26409")

['InboundCustomerService_29884',
 'TarenBuckwalterPrince_23493',
 'JazzminJohnson_20476',
 'ChauniquePrentiss_21170',
 'YurisDelcid_22677',
 'Caregiver_35924',
 'JenniferLy_41314',
 'Cashier_3529',
 'OperationsProcessor_30149',
 'ShiftLeader_16570']